In [1]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180

%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import matplotlib.cm as cm
from matplotlib import gridspec
from scipy import signal
from itertools import combinations
import matplotlib.patches as mpatches

import numpy as np
import os
import shutil
import cv2

from tqdm import trange, tqdm

#
#from numba import jit
#from numba.experimental import jitclass
from numba import jit
#from numba import generated_jit, types

# 
from scipy import signal

#    
from scipy.io import loadmat
import scipy

# 
import csv

from Analyze import (plot_pairwise_interactions, 
                     plot_pairwise_interactions2, 
                     ethogram_social_huddling)
                    


Autosaving every 180 seconds


In [2]:
###############################################
###############################################
###############################################

#
fname = '/media/cat/256GB/dan/cohort2/cluster_slp_files/track_all_day_[0, 1, 2, 3, 4, 5].npy'
locs = np.load(fname)#.astype('float64')
print (locs.shape)

# binarize data into grid world of 100 pixels 
bin_size = 100

#
locs_binned = locs//bin_size
print (locs_binned.shape)

#



(1872130, 6, 2)
(1872130, 6, 2)


/media/cat/4TBSSD/anaconda3/envs/gerbil/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in floor_divide
  


In [3]:
###############################################
########### TRY IDENTITY FREE FIRST ###########
###############################################

@jit
def get_binarized_movies_pooled_mice(locs_binned,
                                     frame_grid):
    
    ''' Here we don't care about mice identity
    '''

    # 
    for k in range(locs_binned.shape[0]):
        #
        for a in range(locs_binned.shape[1]):
            if np.isnan(locs_binned[k][a]).sum()==0:
                x= int(locs_binned[k][a][0])
                y= int(locs_binned[k][a][1])
                frame_grid[k,
                           x, 
                           y] = 1

    return frame_grid

#
# make a 12 x 12 grid to start
frame_grid = np.zeros((locs_binned.shape[0],12,12),dtype='float32')

#
frame_grid = get_binarized_movies_pooled_mice(locs_binned, 
                                  frame_grid)
print ("DONE...")
print ("frame_grid: ", frame_grid.shape)

#################################
#################################
#################################
plt.figure()
for k in range(100):
    ax=plt.subplot(10,10,k+1)
    idx = np.random.choice(frame_grid.shape[0])
    #idx = k*1000
    plt.imshow(frame_grid[idx]
               )
    plt.xticks([])
    plt.yticks([])
    plt.title("# "+str(idx),
             fontsize=10,
             pad=0.9)
plt.show()


DONE...
frame_grid:  (1872130, 12, 12)


In [4]:
##########################################
############### TSNE #####################
##########################################

from openTSNE import TSNE

#
X = frame_grid.reshape(frame_grid.shape[0], -1)

#
tsne = TSNE(
    perplexity=30,
    metric="euclidean",
    n_jobs=16,
    random_state=42,
    verbose=True,
)

embedding_tsne = tsne.fit(X[::10])

print ("embedding: ", embedding_tsne.shape)

--------------------------------------------------------------------------------
TSNE(n_jobs=16, random_state=42, verbose=True)
--------------------------------------------------------------------------------
===> Finding 90 nearest neighbors using Annoy approximate search using euclidean distance...
   --> Time elapsed: 39.89 seconds
===> Calculating affinity matrix...
   --> Time elapsed: 4.69 seconds
===> Calculating PCA-based initialization...
   --> Time elapsed: 1.60 seconds
===> Running optimization with exaggeration=12.00, lr=15601.08 for 250 iterations...
Iteration   50, KL divergence 6.9323, 50 iterations in 10.5187 sec
Iteration  100, KL divergence 6.1339, 50 iterations in 10.6761 sec
Iteration  150, KL divergence 5.8655, 50 iterations in 10.6589 sec
Iteration  200, KL divergence 5.7335, 50 iterations in 10.4239 sec
Iteration  250, KL divergence 5.6427, 50 iterations in 10.6449 sec
   --> Time elapsed: 52.92 seconds
===> Running optimization with exaggeration=1.00, lr=15601.

In [ ]:
import umap
fit = umap.UMAP()
u = fit.fit_transform(X[::10])

print ("Umap res: ", u.shape)

/media/cat/4TBSSD/anaconda3/envs/gerbil/lib/python3.7/site-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)


In [12]:
#######################################################
############### TSNE VISUALIZE AND CLUSTER ############
#######################################################
from sklearn.cluster import DBSCAN
#clustering = DBSCAN(eps=1, 
#                    min_samples=2).fit(embedding_train)

#
n_clusters = np.unique(clustering.labels_)
print ("# clusters: ", n_clusters)

#
clrs = np.zeros(embedding_train.shape[0])
min_size = 500
tsne_locs = []
tsne_clrs = []
tsne_ids = []
ctr=0
for k in n_clusters:
    idx = np.where(clustering.labels_==k)[0]
    
    if idx.shape[0]>=min_size:
        tsne_locs.append(embedding_train[idx])
        tsne_clrs.append(np.zeros(idx.shape[0])+ctr)
        tsne_ids.append(idx)
        ctr+=1
     
tsne_locs = np.vstack(tsne_locs)
print (tsne_locs.shape)
tsne_clrs = np.hstack(tsne_clrs)
print ("# unique clusters left: ", np.unique(tsne_clrs).shape)

################################

plt.figure()
plt.scatter(tsne_locs[:,0],
            tsne_locs[:,1],
            c=tsne_clrs,
            cmap=cm.gist_ncar)

plt.show()


# clusters:  [ -1   0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16
  17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34
  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52
  53  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70
  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88
  89  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106
 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124
 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142
 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160
 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178
 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196
 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214
 215 216 217 218 219 220 221 222 223]
(164622, 2)
# unique clusters left:  (54,)


In [18]:
################################
grid_subsampled = X[::10]
print ("grid_subsampled: ", grid_subsampled.shape)

plt.figure()
for k in range(60):
    try:
        ax=plt.subplot(8,8,k+1)

        idx_local = tsne_ids[k]

        temp = grid_subsampled[idx_local].reshape(idx_local.shape[0], 12,12)
        #print (temp.shape)

        plt.imshow(temp.mean(0))

        plt.xticks([])
        plt.yticks([])
        plt.title("# "+str(idx_local.shape[0]),
             fontsize=10,
             pad=0.9)
    except:
        break
    
plt.show()

grid_subsampled:  (187213, 144)


In [20]:
#########################################
#########################################
#########################################

plt.figure()
plt.scatter(embedding_train[:,0],
            embedding_train[:,1],
            c=np.arange(embedding_train.shape[0]),
            cmap=cm.viridis)

plt.show()

In [35]:
from sklearn.cluster import DBSCAN
clustering = DBSCAN(eps=1, 
                    min_samples=2).fit(embedding_train)

#
n_clusters = np.unique(clustering.labels_)
print ("# clusters: ", n_clusters)

#
clrs = np.zeros(embedding_train.shape[0])
for k in n_clusters:
    idx = np.where(clustering.labels_==k)[0]
    clrs[idx]=k

#############################################
plt.figure()
plt.scatter(embedding_train[:,0],
            embedding_train[:,1],
            c=clrs,
            cmap=cm.gist_ncar)

plt.show()pca_data